<a href="https://colab.research.google.com/github/rendalamili/ml-for-table-extraction/blob/main/Final_Metrics_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy

In [211]:
import os
from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
# Upload zip file containing both ground truth and output csvs - metrics.zip
uploaded = files.upload()

Saving metrics.zip to metrics.zip


In [4]:
# Extract zip
with zipfile.ZipFile('metrics.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [9]:
# Ref: https://stackoverflow.com/questions/4623446/how-do-you-sort-files-numerically/4623518#4623518
import re

def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)

In [131]:
def match_files(n):
    output_dir = '/content/output_csvs/'
    ground_truth_dir = '/content/ground_truth_csvs/'
    # Get sorted list of files
    output_files = os.listdir(output_dir)
    sort_nicely(output_files)

    ground_truth_files = os.listdir(ground_truth_dir)
    sort_nicely(ground_truth_files)

    # match files based on first 10 characters
    matched_files = []
    for output_file in output_files:
      output_name = output_file[:10]
      ground_truth_file = None
      for ground_truth_file in ground_truth_files:
        if ground_truth_file[:10] == output_name:
          matched_files.append((output_file, ground_truth_file))
    return(matched_files[n-1])

In [145]:
def load_output_dfs(file):
    output_dir = '/content/output_csvs/'
    output_csv = pd.read_csv(output_dir + file)
    output_df = pd.DataFrame(output_csv)
    return output_df
df = load_output_dfs(match_files(1)[0])
df

,Form,Average,retail price,Preparation yield factor,Size ec,of a cup equivalent,Average price per cup equivalent
0,Fresh1,$1.85,per pound,0.9,0.243,Pounds,$0.50
1,Applesauce?,$1.17,per pound,1.0,0.540,Pounds,$0.63
2,Juice,NaN,NaN,NaN,NaN,NaN,NaN
3,Ready to drink,$0.87,per pint,1.0,8.000,Fluid ounces,$0.43
4,Frozen 4,$0.61,per pint,1.0,8.000,Fluid ounces,$0.30


In [199]:
def format_ground_truth_df(ground_truth_df):
    ## format ground_truth_csv df for accuracy comparison
    # first row should be header, not row[0]
    ground_truth_df.columns = ground_truth_df.iloc[0]
    ground_truth_df = ground_truth_df[1:]
    ground_truth_df

    # Continuously remove rows where all values from column 2 onward are NaN
    while not ground_truth_df.empty:
        if ground_truth_df.iloc[-1, 1:].isna().all():
            # Drop the last row
            ground_truth_df = ground_truth_df.iloc[:-1]
        else:
            break  # Stop when a valid row is found
    return ground_truth_df

In [200]:
def load_ground_truth_dfs(file):
    # Read ground_truth_csv 1 as df
    ground_truth_dir = '/content/ground_truth_csvs/'
    ground_truth_csv =pd.read_csv(ground_truth_dir + file, encoding='unicode_escape')
    ground_truth_df = pd.DataFrame(ground_truth_csv)
    ground_truth_df = format_ground_truth_df(ground_truth_df)
    return ground_truth_df
df2 = load_ground_truth_dfs(match_files(1)[1])
df2


,Form,Average retail price,NaN,Preparation yield factor,Size of a cup equivalent,NaN,Average price per cup equivalent
1,Fresh1,$1.85,per pound,0.9,0.243,Pounds,$0.50
2,Applesauce2,$1.17,per pound,1,0.540,Pounds,$0.63
3,Juice,NaN,NaN,NaN,NaN,NaN,NaN
4,Ready to drink3,$0.87,per pint,1,8,Fluid ounces,$0.43
5,Frozen4,$0.61,per pint,1,8,Fluid ounces,$0.30


In [193]:
def structure_accuracy(output_df, ground_truth_df):
    # Structure metrics - no. of rows and columns match
    structure_metrics = {
        "row_count_match": output_df.shape[0] == ground_truth_df.shape[0],
        "column_count_match": output_df.shape[1] == ground_truth_df.shape[1]}
    structure_accuracy = sum(structure_metrics.values()) / len(structure_metrics)
    return structure_accuracy

In [ ]:
y_true = ground_truth.df.values.flatten()
y_pred = output.df.values.flatten()

cell

In [213]:
def row_accuracy(output_df, ground_truth_df):
    matching_rows = sum(ground_truth_df.iloc[i].equals(output_df.iloc[i]) for i in range(len(ground_truth_df)))
    row_accuracy = matching_rows / len(ground_truth_df)
    return row_accuracy



In [218]:
for n in range(1,46):
    output_df = load_output_dfs(match_files(n)[0])
    ground_truth_df = load_ground_truth_dfs(match_files(n)[1])
    row = row_accuracy(output_df, ground_truth_df)
    print(f"ROW ACCURACY FOR FILE {n}: {row:.2%}")

IndexError: single positional indexer is out-of-bounds

In [ ]:
def cell_accuracy(ouput_df, ground_truth_df):
    return 0

In [185]:
def detailed_results(n):
    output_df = load_output_dfs(match_files(n)[0])
    ground_truth_df = load_ground_truth_dfs(match_files(n)[1])
    print("OUTPUT DF", n, output_df)
    print("GROUND TRUTH DF", n, ground_truth_df)
    print(f"STRUCTURE ACCURACY FOR FILE {n}: {structure_accuracy(output_df, ground_truth_df):.2%}")

In [183]:
def all_detailed_results():
    for n in range(1,46):
        detailed_results(n)
        print("\n" + "-"*50 + "\n")

In [220]:
def results_df():
    results = []
    for n in range(1,46):
        output_file = match_files(n)[0]
        ground_truth_file = match_files(n)[1]
        output_df = load_output_dfs(output_file)
        ground_truth_df = load_ground_truth_dfs(ground_truth_file)
        results.append({
            'Output File': output_file,
            'Ground Truth File': ground_truth_file,
            'Structure Accuracy': structure_accuracy(output_df, ground_truth_df),
            'Row Accuracy': 0,
            'Cell Accuracy': 0
            #'Row Accuracy': row_accuracy(output_df, ground_truth_df),
            #'Cell Accuracy': cell_accuracy(output_df, ground_truth_df)
        })
    pd.options.display.float_format = '{:.2%}'.format
    return pd.DataFrame(results)

results_df()

,Output File,Ground Truth File,Structure Accuracy,Row Accuracy,Cell Accuracy
0,table_1_Apples-2022_page_1_table_1.csv,table_1_Apples-2022.csv,100.00%,0,0
1,table_2_Apricots-2022_page_1_table_1.csv,table_2_Apricots-2022.csv,100.00%,0,0
2,table_3_Artichoke-2022_page_1_table_1.csv,table_3_Artichoke-2022.csv,100.00%,0,0
3,table_4_Beets-2022_page_1_table_1.csv,table_4_Beets-2022.csv,100.00%,0,0
4,table_5_Black-beans-2022_page_1_table_1.csv,table_5_Black-beans-2022.csv,100.00%,0,0
5,table_6_Blackberries-2022_page_1_table_1.csv,table_6_Blackberries-2022.csv,100.00%,0,0
6,table_7_Blueberries-2022_page_1_table_1.csv,table_7_Blueberries-2022.csv,100.00%,0,0
7,table_8_Cabbage-2022_page_1_table_1.csv,table_8_Cabbage-2022.csv,100.00%,0,0
8,table_9_Cantaloupe-2022_page_1_table_1.csv,table_9_Cantaloupe-2022.csv,0.00%,0,0
9,table_10_Carrots-2022_page_1_table_1.csv,table_10_Carrots-2022.csv,100.00%,0,0
